# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Apr 19 2023 9:09AM,261387,10,38102,"Digital Brands, LLC",Released
1,Apr 19 2023 9:09AM,261387,10,38103,"Digital Brands, LLC",Released
2,Apr 19 2023 9:09AM,261387,10,38104,"Digital Brands, LLC",Released
3,Apr 19 2023 9:09AM,261387,10,38105,"Digital Brands, LLC",Released
4,Apr 19 2023 9:09AM,261387,10,38106,"Digital Brands, LLC",Released
5,Apr 19 2023 9:09AM,261387,10,38107,"Digital Brands, LLC",Released
6,Apr 19 2023 9:09AM,261387,10,38108,"Digital Brands, LLC",Released
7,Apr 19 2023 9:09AM,261387,10,38109,"Digital Brands, LLC",Released
8,Apr 19 2023 8:57AM,261386,10,SONSB0002176,"Nextsource Biotechnology, LLC",Released
9,Apr 19 2023 8:56AM,261385,10,PRF-58684,Bio-PRF,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
15,261383,Released,8
16,261384,Released,1
17,261385,Released,10
18,261386,Released,1
19,261387,Released,8


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
261383,NaN,8.0
261384,NaN,1.0
261385,NaN,10.0
261386,NaN,1.0
261387,NaN,8.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
260869,0.0,33.0
260910,6.0,19.0
261117,0.0,10.0
261300,0.0,1.0
261331,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
260869,0,33
260910,6,19
261117,0,10
261300,0,1
261331,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,260869,0,33
1,260910,6,19
2,261117,0,10
3,261300,0,1
4,261331,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,260869,,33
1,260910,6,19
2,261117,,10
3,261300,,1
4,261331,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Apr 19 2023 9:09AM,261387,10,"Digital Brands, LLC"
8,Apr 19 2023 8:57AM,261386,10,"Nextsource Biotechnology, LLC"
9,Apr 19 2023 8:56AM,261385,10,Bio-PRF
19,Apr 19 2023 8:50AM,261384,10,Emerginnova
20,Apr 19 2023 8:45AM,261383,10,Hush Hush
28,Apr 19 2023 8:40AM,261382,10,Emerginnova
30,Apr 19 2023 8:39AM,261381,10,"Uniderm USA, Inc"
31,Apr 19 2023 8:18AM,261117,20,Reliable 1 Laboratories LLC
41,Apr 19 2023 8:10AM,261378,19,ACG North America LLC
42,Apr 19 2023 8:00AM,261377,19,"AdvaGen Pharma, Ltd"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Apr 19 2023 9:09AM,261387,10,"Digital Brands, LLC",,8
1,Apr 19 2023 8:57AM,261386,10,"Nextsource Biotechnology, LLC",,1
2,Apr 19 2023 8:56AM,261385,10,Bio-PRF,,10
3,Apr 19 2023 8:50AM,261384,10,Emerginnova,,1
4,Apr 19 2023 8:45AM,261383,10,Hush Hush,,8
5,Apr 19 2023 8:40AM,261382,10,Emerginnova,,2
6,Apr 19 2023 8:39AM,261381,10,"Uniderm USA, Inc",,1
7,Apr 19 2023 8:18AM,261117,20,Reliable 1 Laboratories LLC,,10
8,Apr 19 2023 8:10AM,261378,19,ACG North America LLC,,1
9,Apr 19 2023 8:00AM,261377,19,"AdvaGen Pharma, Ltd",,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Apr 19 2023 9:09AM,261387,10,"Digital Brands, LLC",8,
1,Apr 19 2023 8:57AM,261386,10,"Nextsource Biotechnology, LLC",1,
2,Apr 19 2023 8:56AM,261385,10,Bio-PRF,10,
3,Apr 19 2023 8:50AM,261384,10,Emerginnova,1,
4,Apr 19 2023 8:45AM,261383,10,Hush Hush,8,
5,Apr 19 2023 8:40AM,261382,10,Emerginnova,2,
6,Apr 19 2023 8:39AM,261381,10,"Uniderm USA, Inc",1,
7,Apr 19 2023 8:18AM,261117,20,Reliable 1 Laboratories LLC,10,
8,Apr 19 2023 8:10AM,261378,19,ACG North America LLC,1,
9,Apr 19 2023 8:00AM,261377,19,"AdvaGen Pharma, Ltd",1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Apr 19 2023 9:09AM,261387,10,"Digital Brands, LLC",8,
1,Apr 19 2023 8:57AM,261386,10,"Nextsource Biotechnology, LLC",1,
2,Apr 19 2023 8:56AM,261385,10,Bio-PRF,10,
3,Apr 19 2023 8:50AM,261384,10,Emerginnova,1,
4,Apr 19 2023 8:45AM,261383,10,Hush Hush,8,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Apr 19 2023 9:09AM,261387,10,"Digital Brands, LLC",8.0,NaN
1,Apr 19 2023 8:57AM,261386,10,"Nextsource Biotechnology, LLC",1.0,NaN
2,Apr 19 2023 8:56AM,261385,10,Bio-PRF,10.0,NaN
3,Apr 19 2023 8:50AM,261384,10,Emerginnova,1.0,NaN
4,Apr 19 2023 8:45AM,261383,10,Hush Hush,8.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Apr 19 2023 9:09AM,261387,10,"Digital Brands, LLC",8.0,0.0
1,Apr 19 2023 8:57AM,261386,10,"Nextsource Biotechnology, LLC",1.0,0.0
2,Apr 19 2023 8:56AM,261385,10,Bio-PRF,10.0,0.0
3,Apr 19 2023 8:50AM,261384,10,Emerginnova,1.0,0.0
4,Apr 19 2023 8:45AM,261383,10,Hush Hush,8.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2352370,32.0,1.0
12,261368,1.0,0.0
19,1045432,30.0,0.0
20,1044196,63.0,6.0
22,261370,0.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2352370,32.0,1.0
1,12,261368,1.0,0.0
2,19,1045432,30.0,0.0
3,20,1044196,63.0,6.0
4,22,261370,0.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,32.0,1.0
1,12,1.0,0.0
2,19,30.0,0.0
3,20,63.0,6.0
4,22,0.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,32.0
1,12,Released,1.0
2,19,Released,30.0
3,20,Released,63.0
4,22,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,19,20,22
Status,,,,,
Executing,1.0,0.0,0.0,6.0,1.0
Released,32.0,1.0,30.0,63.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,19,20,22
0,Executing,1.0,0.0,0.0,6.0,1.0
1,Released,32.0,1.0,30.0,63.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,19,20,22
0,Executing,1.0,0.0,0.0,6.0,1.0
1,Released,32.0,1.0,30.0,63.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()